In [1]:
import scanpy as sc
import scvi
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import coo_matrix

Global seed set to 0


In [2]:
tox_adata = sc.read_h5ad("/project/gilad/jpopp/cellcipher/data/Tox1-full/single_cell_objects/filtered_pflog1ppfnorm.hvg.umap_embedding.h5ad")
tox_adata_raw = sc.read_h5ad("/project/gilad/jpopp/cellcipher/data/Tox1-full/single_cell_objects/filtered.h5ad")

In [3]:
dmso_cells = [tox_adata.obs.index[i] for i in range(tox_adata.obs.shape[0]) if tox_adata.obs['sample_id'][i] in ['YG-KR-10X-13s-Tox-01', 'YG-KR-10X-13s-Tox-02']]

In [4]:
dmso_adata = tox_adata[dmso_cells]
dmso_adata_raw = tox_adata_raw[dmso_cells]

Split counts (Poisson count splitting, for now)

In [5]:
dmso_coo_train = coo_matrix(dmso_adata_raw.X.copy())
dmso_coo_test = coo_matrix(dmso_adata_raw.X.copy())
dmso_nonzero = dmso_coo_train.data

In [7]:
dmso_nonzero_train = np.random.binomial(n=dmso_nonzero.astype(int), p=0.5)
dmso_nonzero_test = dmso_nonzero - dmso_nonzero_train

In [8]:
dmso_coo_train.data = dmso_nonzero_train
dmso_train = dmso_coo_train.tocsr()
dmso_train.eliminate_zeros()

dmso_coo_test.data = dmso_nonzero_test
dmso_test = dmso_coo_test.tocsr()
dmso_test.eliminate_zeros()

In [9]:
dmso_adata_raw.layers['train'] = dmso_train
dmso_adata_raw.layers['test'] = dmso_test

In [10]:
# Set up anndata object for scVI
scvi.model.LinearSCVI.setup_anndata(dmso_adata_raw, layer='train')

# Train linearly decoded VAE model
mod = scvi.model.LinearSCVI(dmso_adata_raw, n_latent=10, 
                            gene_likelihood = 'nb', latent_distribution='ln')
mod.train(max_epochs=100, plan_kwargs={'lr':5e-3}, check_val_every_n_epoch=10)

# Save results
dmso_adata_raw.obsm["X_scVI"] = mod.get_latent_representation()

/scratch/midway2/jpopp/.conda/envs/scvi-scanpy/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=100 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 100/100: 100%|██████████| 100/100 [13:54<00:00,  8.34s/it, loss=5.32e+03, v_num=1]


In [13]:
dmso_adata_test = dmso_adata_raw.copy()

In [20]:
dmso_adata_raw.obsm['X_scVI']

array([[0.0248289 , 0.11522806, 0.09170242, ..., 0.13460532, 0.11373047,
        0.10688931],
       [0.02881909, 0.15218505, 0.0571594 , ..., 0.10337201, 0.19204408,
        0.03881453],
       [0.04336597, 0.0187551 , 0.12689348, ..., 0.13605908, 0.28828502,
        0.09914818],
       ...,
       [0.06926972, 0.11502334, 0.07196866, ..., 0.00257266, 0.14235353,
        0.13015486],
       [0.07543416, 0.08032739, 0.1347192 , ..., 0.10344084, 0.1099667 ,
        0.09161605],
       [0.05709597, 0.08106097, 0.02911263, ..., 0.02973681, 0.20784071,
        0.13112696]], dtype=float32)

In [24]:
mod.get_likelihood_parameters()

AttributeError: 'NegativeBinomial' object has no attribute 'zi_probs'